In [2]:
%load_ext autoreload
%autoreload 2

import sys
import os
import logging

import numpy as np
import argparse
import copy
import pandas as pd
import shutil
import json
import urllib

from   datetime import datetime, timedelta

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import InvalidRequestError, IntegrityError 
from sqlalchemy.orm.exc import NoResultFound

from matplotlib import pyplot as plt

from tradingdb.utils import *
from tradingdb.ORM.utils import *

logger = logging.getLogger()
logger.setLevel(logging.INFO)
# create file handler which logs even debug messages
handler = logging.StreamHandler()
handler.setLevel(logging.NOTSET)
format = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
handler.setFormatter(format)


logger.addHandler(handler)
handler.setLevel(logging.DEBUG)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
DBNAME=os.path.abspath("../data/master.db")

In [4]:
session=get_new_session(DBNAME)


In [62]:
intervals=['1d','1h']
def get_last_price_for_period(stock):
    period=[]
    date=[]
    for p in stock.prices:
        date.append(p.date)
        period.append(p.period.name)
    periods=set(period)
    df=pd.DataFrame({'date':date,'period':period})
    return {pp:df[df['period'] == pp]['date'].max().to_pydatetime() for pp in periods}


stock_last_date={}
with session.begin() as mysession:
    q=mysession.query(stock.Stock)
    #q=q.join(stock.Exchange)
    #q=q.join(stock.Price)
    #q=q.join(stock.Period)
    #q=q.filter(stock.Period=='1d')
    q=q.all()
    #symbols=[ r.yf_symbol for r in q ]
    for r in q:
        stock_last_date[r.yf_symbol]=get_last_price_for_period(r)
        #print(r.company)
        #print(r.prices)
        #break

In [63]:
# lets say 1 week
deltadays=14
period='1h'

def add_dfd_to_yfsymbol(session,interval,dfd):
    with session.begin() as mysession:
        for k,df in dfd.items():
            nadd=0
            for index, row in df.iterrows():
                # convert name into time
                sid=mysession.query(stock.Stock.id).filter(stock.Stock.yf_symbol==k).one()[0]
                pid=mysession.query(stock.Period.id).filter(stock.Period.name==interval).one()[0]
                q=mysession.query(stock.Price)
                q=q.filter(stock.Price.stock_id==sid)
                q=q.filter(stock.Price.period_id==pid)
                r=q.filter(stock.Price.date==index.to_pydatetime()).one_or_none()
                if r is None:
                    #print("adding time ",index)
                    p=stock.Price(
                                stock_id=sid,
                                period_id=pid,
                                date=index.to_pydatetime(),  
                                open=row['Open'],
                                low=row['Low'],
                                high=row['High'],
                                close=row['Close'],
                                adjusted_close=row['Adj Close'],
                                volume=row['Volume'])
                    mysession.add(p)
                    nadd+=1
                #else:
                    #print("time ",index,"already in")
            print("Symbol ",k,": added ",nadd," timepoints for interval ",interval)

for k,v in stock_last_date.items():
    #print(k,v)
    time_end=v[period]+timedelta(days=deltadays)
    time_start = v[period]
    #print(time_start,time_end)
    # fetch this
    dfd=download_data_from_yfsymbols(k,time_start,time_end,interval=period)
    # add this
    add_dfd_to_yfsymbol(session,period,dfd)
    #break

[*********************100%***********************]  1 of 1 completed
Symbol  JD.L : added  89  timepoints for interval  1h
[*********************100%***********************]  1 of 1 completed
Symbol  CCH.L : added  89  timepoints for interval  1h
[*********************100%***********************]  1 of 1 completed
Symbol  FRES.L : added  89  timepoints for interval  1h
[*********************100%***********************]  1 of 1 completed
Symbol  SMT.L : added  89  timepoints for interval  1h
[*********************100%***********************]  1 of 1 completed
Symbol  NXT.L : added  89  timepoints for interval  1h
[*********************100%***********************]  1 of 1 completed
Symbol  RR.L : added  89  timepoints for interval  1h
[*********************100%***********************]  1 of 1 completed
Symbol  ADM.L : added  89  timepoints for interval  1h
[*********************100%***********************]  1 of 1 completed
Symbol  MRO.L : added  89  timepoints for interval  1h
[********